In [1]:
from ultralytics import YOLO

model = YOLO('../yolo11n-pose.pt')

model.export(
    format='engine',
    half=True,
    imgsz=640,
    workspace=8,
    # batch=8,
    # int8=True,
    # data='coco-pose.yaml',
)

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.20 🚀 Python-3.11.10 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLO11n-pose summary (fused): 257 layers, 2,866,468 parameters, 0 gradients, 7.4 GFLOPs

PyTorch: starting from '../yolo11n-pose.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 56, 8400) (6.0 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.35...
ONNX: export success ✅ 1.4s, saved as '../yolo11n-pose.onnx' (11.3 MB)

TensorRT: starting export with TensorRT 10.3.0...
[10/29/2024-01:39:23] [TRT] [I] [MemUsageChange] Init CUDA: CPU +1, GPU +0, now: CPU 794, GPU 1190 (MiB)
[10/29/2024-01:39:32] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +2090, GPU +386, now: CPU 3040, GPU 1576 (MiB)
[10/29/2024-01:39:32] [TRT] [I] ----------------------------------------------------------------
[10/29/2024-01:39:32] [TRT] [I] Input filename:   ../yolo11n-pose.onnx

'../yolo11n-pose.engine'

# 3x speedup

In [10]:
import cv2
tensorrt_model = YOLO('../yolo11n-pose.engine')
video_path = '../fall_dataset/videos/video_2.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened:
    print("Error could not open video")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    results = tensorrt_model(frame)
    for result in results:
        frame = result.plot()

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Loading yolo11n-pose.engine for TensorRT inference...
[10/29/2024-01:13:31] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[10/29/2024-01:13:31] [TRT] [I] Loaded engine size: 12 MiB
[10/29/2024-01:13:31] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +18, now: CPU 3, GPU 94 (MiB)

0: 640x640 3 persons, 8.3ms
Speed: 2.6ms preprocess, 8.3ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 9.5ms
Speed: 2.0ms preprocess, 9.5ms inference, 6.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 9.5ms
Speed: 1.5ms preprocess, 9.5ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 9.4ms
Speed: 1.7ms preprocess, 9.4ms inference, 6.0ms postprocess per image at shape (1, 3